In [ ]:
import numpy as np
import pandas as pd
df_minites = pd.read_csv(r"/content/drive/MyDrive/miniproject/의사록_한글자제거.csv")
df_minites=df_minites.drop("Unnamed: 0",axis=1)

from ekonlpy.sentiment import MPCK
mpck = MPCK()

def make_ngram(sen):
    tokens=mpck.tokenize(sen)
    ngrams=mpck.ngramize(tokens)
    total_list=tokens+ngrams
    return total_list

df_Pos=pd.read_csv(r"/content/drive/MyDrive/miniproject/긍정사전except1.csv")
df_Neg=pd.read_csv(r"/content/drive/MyDrive/miniproject/부정사전except1.csv")

Pos_list=df_Pos['ngram'].to_list()
Neg_list=df_Neg['ngram'].to_list()

#df_minites=df_minites.drop(['up_count','down_count'],axis=1)

df_minites['pos_count']=np.nan
df_minites['neg_count']=np.nan

for i in range(len(df_minites)):
    #의사록 어조
    pos_count=0
    neg_count=0
    sentences=df_minites['content'][i].split("@@@")
    for sentence in sentences:
        #문장 어조 
        up_count=0
        down_count=0
        ngram_list=make_ngram(sentence)
        for item in ngram_list:
            if item in Pos_list:
                up_count=up_count+1
            elif item in Neg_list:
                down_count=down_count+1
        try:
            #print(up_count, down_count)
            sentence_tone = (up_count-down_count)/(up_count+down_count)
        except Exception as e:
            #print(e)
            sentence_tone=0
        if sentence_tone >0:
            pos_count=pos_count+1
        elif sentence_tone<0:
            neg_count=neg_count+1
    df_minites.loc[i, ['pos_count']]=pos_count
    df_minites.loc[i, ['neg_count']]=neg_count
    print(i)

df_minites.to_csv(r"/content/drive/MyDrive/miniproject/munute_2final_except1_2.csv", encoding='utf-8-sig', index=False)

2008~ 2021 기준 금리

In [ ]:
#기준 금리 파일 수기로 작성
date = ['2007.08.09', '2008.08.07', '2008.10.09', '2008.10.27','2008.11.07', '2008.12.11', '2009.01.09', '2009.02.12', '2010.07.09',
        '2010.11.16', '2011.01.13', '2011.03.10', '2011.06.10', '2012.07.12', '2012.10.11', '2013.05.09', '2014.08.14', '2014.10.15',
        '2015.03.12', '2015.06.11', '2016.06.09', '2017.11.30', '2018.11.30', '2019.07.18', '2019.10.16', '2020.03.17', '2020.05.28', '2021.08.26', '2021.11.25']
standardrate = [5.00, 5.25, 5.00, 4.25, 4.00, 3.00, 2.50, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.00, 2.75, 2.50, 2.25,
                2.00, 1.75, 1.50, 1.25, 1.50, 1.75, 1.50, 1.25, 0.75, 0.50, 0.75, 1.00]

sr = pd.DataFrame({ "date" : date, "standardrate" : standardrate})
sr.to_csv(r"/content/drive/MyDrive/miniproject/standardrate.csv", encoding='utf-8-sig', index=False)

In [ ]:
import pandas as pd

df=pd.read_csv(r"/content/drive/MyDrive/miniproject/munute_2final_except1_2.csv")

df['TONE_doc']=(df['pos_count']-df['neg_count'])/(df['pos_count']+df['neg_count'])

def get_label(df):
    if df['TONE_doc'] > 0:
        return "hawk"
    elif df['TONE_doc'] < 0:
        return "dove"
        
df['TONE_Doc_Label']=df.apply(get_label,axis=1)
#문서의 톤을 기준으로 매파, 비둘기파 라벨링

df.to_csv("/content/drive/MyDrive/miniproject/munute_2tonefinal_except1_2.csv", encoding='utf-8-sig', index=False)

In [ ]:
# 상관관계 측정을 위해 필요없는 columns 제거
df = df
del df['content']
del df['ngrams']
del df['pos_count']
del df['neg_count']
del df['TONE_doc']
del df['TONE_Doc_Label']
df

In [ ]:
def all_date_range_df(date_range):
    adr = pd.DataFrame({'date':date_range})
    return adr
# 의사록파일의 날짜에 맞는 기준금리 할당된 프레임 작성
date_range = (pd.date_range(start='20080101', end='20211231')).strftime("%Y.%m.%d").tolist()
adr = all_date_range_df(date_range)
adr

In [ ]:
adr2 = pd.merge(df, adr, how='outer',on='date')
adr3 = adr2.sort_values(by='date')
adr3.reset_index(inplace = True, drop = True)

In [ ]:
for i in range(len(adr3)):
    if pd.isna(adr3['standardrate'][i])==True:
        adr3['standardrate'][i] = adr3['standardrate'][i-1]
adr3 # 의사록 날짜, 해당 기간의 기준금리

In [ ]:
import pandas as pd
dfdf1 = pd.read_csv(r'/content/drive/MyDrive/miniproject/2008standardrate.csv', encoding='utf-8') # 기준금리파일
dfdf2 = pd.read_csv(r"/content/drive/MyDrive/miniproject/munute_2tonefinal_except1_2.csv", encoding='utf-8') #전체 합본

In [ ]:
subset = dfdf2[['date', 'TONE_doc']]               #dfdf2데이터프레임에서 날짜와 톤만 추출하는 데이터프레임 만들기
cor_df = pd.merge(subset, adr3, how='inner', on='date') # date를 기준으로 inner조인하여 머지하기
cor_df.corr()